# This file is used to flat the JSON data

In [2]:
import json
import pandas as pd
import numpy as np
from shapely.geometry import Point
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5
import re
import math
import ast
from pandas.io.json import json_normalize
from flatten_json import flatten
import ProjectLookUpTable 
import socket, struct
from binascii import hexlify
import datetime
import macaddress
import os

# Read JSON data

In [3]:
def file_json_file_with_file_name(pcap_Json_file_name):
    json_data_file = open(pcap_Json_file_name, encoding='utf-8')
    file_data = json.load(json_data_file)
    json_data_file.close()
    return file_data

# type_map_for_json(json_record) map string value to int 

In [4]:
# type_map_for_json(json_record)


def IPV6_to_int(ipv6_addr):
    return int(hexlify(socket.inet_pton(socket.AF_INET6, ipv6_addr)), 16)

def IPV4_to_int(ip):
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]

def type_map_for_json(json_record):
    if type(json_record) != dict:
        return True
    for json_key in json_record.keys():
        if type_map_for_json(json_record[json_key]):
            if json_key in ProjectLookUpTable.wire_shark_type_lookup_map:
                json_value_type = ProjectLookUpTable.wire_shark_type_lookup_map[json_key]
                json_str_value = json_record[json_key]
                # change number string to number 
                if isinstance(json_str_value, int) or isinstance(json_str_value, float):
                    json_record[json_key] = json_value_type
                elif json_value_type in ProjectLookUpTable.wire_shark_number_type_set:
                    try:
                        if '0x' in json_str_value:
                            json_record[json_key] = int(json_str_value, 0)
                        elif '.' in json_str_value:
                            json_record[json_key] = float(json_str_value)
                        else:
                            json_record[json_key] = int(json_str_value)
                    except:
                        json_record[json_key] = 0
                elif json_value_type in ProjectLookUpTable.wire_shark_ipaddr_type_set:
                    try:
                        if ':' in json_str_value:
                            json_record[json_key] = IPV6_to_int(json_str_value)
                        else:
                            json_record[json_key] = IPV4_to_int(json_str_value)
                    except:
                        json_record[json_key] = json_str_value
                elif json_value_type == 'Ethernet or other MAC address':
                    try:
                        json_record[json_key] = int(macaddress.MAC(json_str_value))
                    except:
                        json_record[json_key] = json_str_value
                # elif json_value_type == 'Date and time':
                #     json_str_value = json_str_value[0:json_str_value.index('.')+6]
                #     json_record[json_key] = datetime.datetime.strptime(json_str_value, '%b %d, %Y %H:%M:%S.%f')





In [81]:
def check_json_field_exist(json_record, local_wanted_packet_field_pair):
    if len(local_wanted_packet_field_pair) == 0:
        return True
    if local_wanted_packet_field_pair[0] in json_record:
        return local_wanted_packet_field_pair[1] == json_record[local_wanted_packet_field_pair[0]]
    rv = False
    for key in json_record:
        if isinstance(json_record[key], dict):
            rv = rv or check_json_field_exist(json_record[key], local_wanted_packet_field_pair)
    return rv


In [79]:
def get_wanted_packets(local_wanted_packets, local_json_record_list, local_wanted_packet_field_pair):
    rv = []
    idx = 0
    for record in local_json_record_list:
        if local_wanted_packets[idx] in record['_source']['layers'] and \
        check_json_field_exist(record['_source']['layers'][local_wanted_packets[idx]], local_wanted_packet_field_pair[idx]):
            rv.append(record)
            idx += 1
            if idx >= len(local_wanted_packets):
                return rv
    return rv

# drop use less fields by use drop_json_fields(json_record, field_name_path_list)

In [6]:

def json_field_drop(json_record, path_list):
    if path_list[0] not in json_record:
        return
    if(len(path_list) == 1):
        del json_record[path_list.pop(0)]
    else:
        json_record = json_record[path_list.pop(0)]
        json_field_drop(json_record, path_list)

def drop_json_fields(json_record, field_name_path_list):
    for name_path_str in field_name_path_list:
        json_field_drop(json_record, name_path_str.split('->'))


In [7]:
def start_drop_field_value_mapping(local_json_records, local_json_drop_field_path_list):
    for local_json_record in local_json_records:
        local_json_record = local_json_record['_source']['layers']
        drop_json_fields(local_json_record, local_json_drop_field_path_list)
        type_map_for_json(local_json_record)


In [8]:
def flatten_json_records(json_record_list):
    rv = []
    for record in json_record_list:
        record = record['_source']['layers']
        rv.append(flatten(record))
    return rv

# Only for Hancitor

In [164]:
from chepy import Chepy
def Hancitor_processing(local_key_field, target_http_records):
    contain_all = 0
    try:
        if local_key_field in target_http_records[2]:
            encode_str = target_http_records[2][local_key_field]
            url_set = []
            decode_str =str(Chepy(encode_str).base64_decode().xor(key="z", key_type="utf"))
            url_set.append(decode_str[decode_str.index('http://'):decode_str.index('}')])
            decode_str = decode_str[decode_str.index('}')+1:]
            url_set.append(decode_str[decode_str.index('http://'):decode_str.index('}')])
            print(url_set)
            for offSet in range(0,len(url_set)):
                if "http_http.request.full_uri" in target_http_records[3+offSet] \
                and target_http_records[3+offSet]["http_http.request.full_uri"] == url_set[offSet]:
                    contain_all += 1
            
            target_http_records[2]['Hancitor_url'] = contain_all
            
    except:
        target_http_records[2]['Hancitor_url'] = contain_all
        


In [153]:
raw_url = str(Chepy('NTGMARZAEg4OCkBVVRgVGBkbDhUcCB8eHhMUHVQZFRdVSktKSFQYExQHARZAEg4OCkBVVRgVGBkbDhUcCB8eHhMUHVQZFRdVSktKSAlUGBMUBwEYQBIODgpAVVUYFRgZGw4VHAgfHh4TFB1UGRUXVUwWGwwcHhFUHwIfBw==').base64_decode().xor(key="z", key_type="utf"))


raw_url[raw_url.index('http://'):raw_url.index('}')]
raw_url = raw_url[raw_url.index('}')+1:]
raw_url[raw_url.index('http://'):raw_url.index('}')]

'http://bobcatofredding.com/0102s.bin'

# Create the string to integer map so the ML process can run faster 


In [10]:

def map_str_to_int(local_record_str_to_int_map, local_flatten_json_records_list):
    for flatten_json_record in local_flatten_json_records_list:
        for field_name in flatten_json_record:
            if isinstance(flatten_json_record[field_name], str):
                if flatten_json_record[field_name] not in local_record_str_to_int_map:
                    local_record_str_to_int_map[flatten_json_record[field_name]] = len(local_record_str_to_int_map)+1
                flatten_json_record[field_name] = local_record_str_to_int_map[flatten_json_record[field_name]]



In [11]:
def get_df_header_of_each_col(local_num_record, malware_json_records_list_list, normal_json_records_list_list):
    local_col_name_on_offset = {}

    # init set
    for off_set in range(local_num_record):
        local_col_name_on_offset[off_set] = set()

    # get all unique fiels of each records on each offset
    for local_json_records_list in malware_json_records_list_list:
        for off_set in range(local_num_record):
            if off_set < len(local_json_records_list):
                for json_field in local_json_records_list[off_set]:
                    local_col_name_on_offset[off_set].add(json_field)

    # get all unique fiels of each records on each offset
    for local_json_records_list in normal_json_records_list_list:
        for off_set in range(local_num_record):
            if off_set < len(local_json_records_list):
                for json_field in local_json_records_list[off_set]:
                    local_col_name_on_offset[off_set].add(json_field)

    return local_col_name_on_offset

In [12]:
 # Cast the list of json records into one record and load into panda
def make_list_json_record_into_one_record(local_col_name_on_offset, local_json_records_list_list, local_num_record):

    # use the offset and unique fields name to build df
    local_overall_field_list = []
    for off_set in range(local_num_record):
        for field_name in local_col_name_on_offset[off_set]:
            local_overall_field_list.append(field_name+'_'+str(off_set))
    local_df = pd.DataFrame(columns=local_overall_field_list)
    try:
        # load data into the df
        for local_json_records_list in local_json_records_list_list:
            local_record_array = []
            for off_set in range(local_num_record):
                if off_set < len(local_json_records_list):
                    local_json_record = local_json_records_list[off_set]
                    for df_col_field in local_col_name_on_offset[off_set]:
                        if df_col_field in local_json_record:
                            local_record_array.append(local_json_record[df_col_field])
                        else:
                            local_record_array.append(0)
            local_df.loc[len(local_df.index)] = local_record_array
    except:
        print(local_record_array)
    return local_df



# Main


In [92]:
# important variables 
json_records_list_list = []
file_name_lists = []

for top_file_path in os.listdir("../../data/Hancitor"):
    local_path = '../../data/Hancitor/'+top_file_path + '/only/only.json'
    if os.path.exists(local_path):
        file_name_lists.append(local_path)

wanted_packets = ['http', 'http', 'http', 'http', 'http']
wanted_packet_field_pair = [["http.request.method","GET"], ["http.request.method","POST"], ["http.content_type","text/html",],["http.request.method","GET"],["http.request.method","GET"]]
json_drop_field_path_list = ['frame->frame.time', 'frame->frame.time_delta_displayed', 
    'frame->frame.coloring_rule.name', 'frame->frame.coloring_rule.string',
    'ip->ip.addr', 'ip->ip.host', 'tcp->tcp.port','http->http.request.line',
    'tcp->tcp.payload', 
    'ip->ip.src_host', 'ip->ip.dst_host', 'http->http.cookie_tree',  
    'tls->tls.record->tls.handshake->tls.handshake.random',
    'tls->tls.record->tls.handshake->tls.handshake.random_tree',
    'tls->tls.record->tls.handshake.ciphersuites',
]

record_str_to_int_map = {}

In [93]:
file_name_lists

['../../data/Hancitor/2021-02-02 - QUICK POST HANCITOR INFECTION WITH FICKER STEALER, COBALT STRIKE, & NETSUPPORT RAT/only/only.json',
 '../../data/Hancitor/2021-02-09 (TUESDAY) - QUICK POST HANCITOR INFECTION WITH COBALT STRIKE/only/only.json',
 '../../data/Hancitor/2021-03-18 - HANCITOR (CHANITOR) ACTIVIY (MAN1MOSKALVZAPOETA511)/only/only.json',
 '../../data/Hancitor/2021-04-06 AND 07 (TUESDAY AND WEDNESDAY) - DATA DUMP HANCITOR ACTIVITY/only/only.json',
 '../../data/Hancitor/2021-05-13 (THURSDAY) - HANCITOR WITH FICKER STEALER AND COBALT STRIKE/only/only.json',
 '../../data/Hancitor/2021-05-20 (THURSDAY) - HANCITOR WITH FICKER STEALER, COBALT STRIKE, & NETPING TOOL/only/only.json',
 '../../data/Hancitor/2021-05-24 (MONDAY) - QUICK POST HANCITOR INFECTION WITH FICKER STEALER AND COBALT STRIKE/only/only.json',
 '../../data/Hancitor/2021-06-01 (TUESDAY) - HANCITOR WITH COBALT STRIKE AND NETPING TOOL ACTIVITY/only/only.json',
 '../../data/Hancitor/2021-06-15 (TUESDAY) - QUICK POST HANCI

In [205]:

for  pcap_json_file_name in file_name_lists: 
    # Read Json files
    json_record_list = file_json_file_with_file_name(pcap_json_file_name)
    # make sure the start
    json_record_list = get_wanted_packets(wanted_packets, json_record_list, wanted_packet_field_pair)
    # Drop not wanted fields 
    start_drop_field_value_mapping(json_record_list,json_drop_field_path_list)
    # Flate Json into field value map
    flatten_json_records_list = flatten_json_records(json_record_list)

    # For Hancitor, take the cookie and make them into fields 
    Hancitor_processing("http_http.file_data", flatten_json_records_list)
    
    # Map String into int
    map_str_to_int(record_str_to_int_map, flatten_json_records_list)
    # save current file to list
    json_records_list_list.append(flatten_json_records_list)


[{'_index': 'packets-2021-02-02',
  '_type': 'doc',
  '_score': None,
  '_source': {'layers': {'frame': {'frame.encap_type': '1',
     'frame.time': 'Feb  2, 2021 11:24:20.297567000 Eastern Standard Time',
     'frame.offset_shift': '0.000000000',
     'frame.time_epoch': '1612283060.297567000',
     'frame.time_delta': '0.000000000',
     'frame.time_delta_displayed': '0.000000000',
     'frame.time_relative': '0.000000000',
     'frame.number': '1',
     'frame.len': '218',
     'frame.cap_len': '218',
     'frame.marked': '0',
     'frame.ignored': '0',
     'frame.protocols': 'eth:ethertype:ip:tcp:http',
     'frame.coloring_rule.name': 'HTTP',
     'frame.coloring_rule.string': 'http || tcp.port == 80 || http2'},
    'eth': {'eth.dst': '20:e5:2a:b6:93:f1',
     'eth.dst_tree': {'eth.dst_resolved': 'Netgear_b6:93:f1',
      'eth.dst.oui': '2155818',
      'eth.dst.oui_resolved': 'Netgear',
      'eth.addr': '20:e5:2a:b6:93:f1',
      'eth.addr_resolved': 'Netgear_b6:93:f1',
      '

In [169]:
json_records_list_list

[[{'frame_frame.encap_type': 1,
   'frame_frame.offset_shift': 0.0,
   'frame_frame.time_epoch': 1612283060.297567,
   'frame_frame.time_delta': 0.0,
   'frame_frame.time_relative': 0.0,
   'frame_frame.number': 1,
   'frame_frame.len': 218,
   'frame_frame.cap_len': 218,
   'frame_frame.marked': 0,
   'frame_frame.ignored': 0,
   'frame_frame.protocols': 1,
   'eth_eth.dst': 36168636208113,
   'eth_eth.dst_tree_eth.dst_resolved': 2,
   'eth_eth.dst_tree_eth.dst.oui': 2155818,
   'eth_eth.dst_tree_eth.dst.oui_resolved': 3,
   'eth_eth.dst_tree_eth.addr': 36168636208113,
   'eth_eth.dst_tree_eth.addr_resolved': 2,
   'eth_eth.dst_tree_eth.addr.oui': 2155818,
   'eth_eth.dst_tree_eth.addr.oui_resolved': 3,
   'eth_eth.dst_tree_eth.dst.lg': 0,
   'eth_eth.dst_tree_eth.lg': 0,
   'eth_eth.dst_tree_eth.dst.ig': 0,
   'eth_eth.dst_tree_eth.ig': 0,
   'eth_eth.src': 34395146158,
   'eth_eth.src_tree_eth.src_resolved': 4,
   'eth_eth.src_tree_eth.src.oui': 2050,
   'eth_eth.src_tree_eth.src.ou

# Read and output the normal traffic 

In [196]:
def get_normal_wanted_packets( local_wanted_packets, local_json_list, local_wanted_packet_field_pair):
    local_valid_record_List_group = []
    for local_json_list_idx in range(len(local_json_list)):
        if local_json_list_idx+len(local_wanted_packets)-1 >= len(local_json_list):
            break
        json_record = local_json_list[local_json_list_idx]
        if local_wanted_packets[0] in json_record['_source']['layers'] and local_wanted_packets[0] not in local_json_list[local_json_list_idx+1]['_source']['layers']:
            local_valid_record_List_group.append(get_wanted_packets(wanted_packets, local_json_list[local_json_list_idx:],local_wanted_packet_field_pair))
    return local_valid_record_List_group


In [197]:
normal_json_records_list_list = []

normal_file_name_lists = []
for top_file_path in os.listdir("../../data/Hancitor_normal"):
    if top_file_path.endswith(".json"):
        local_path = '../../data/Hancitor_normal/'+top_file_path
        normal_file_name_lists.append(local_path)

normal_file_name_lists

['../../data/Hancitor_normal/normal1.json',
 '../../data/Hancitor_normal/normal10.json',
 '../../data/Hancitor_normal/normal11.json',
 '../../data/Hancitor_normal/normal2.json',
 '../../data/Hancitor_normal/normal3.json',
 '../../data/Hancitor_normal/normal4.json',
 '../../data/Hancitor_normal/normal5.json',
 '../../data/Hancitor_normal/normal6.json',
 '../../data/Hancitor_normal/normal7.json',
 '../../data/Hancitor_normal/normal8.json',
 '../../data/Hancitor_normal/normal9.json']

In [198]:

for  pcap_json_file_name in normal_file_name_lists: 
    # Read Json files
    json_record_list = file_json_file_with_file_name(pcap_json_file_name)

    # make sure the start
    json_record_list_group = get_normal_wanted_packets(wanted_packets, json_record_list, wanted_packet_field_pair)

    for json_list in json_record_list_group:
        # Drop not wanted fields 
        start_drop_field_value_mapping(json_list,json_drop_field_path_list)
        # Flate Json into field value map
        flatten_json_records_list = flatten_json_records(json_list)
        if len(flatten_json_records_list) < 3:
            continue
        # For Hancitor, take the cookie and make them into fields 
        Hancitor_processing("http_http.file_data", flatten_json_records_list)
        # Map String into int
        map_str_to_int(record_str_to_int_map, flatten_json_records_list)
        # save current file to list
        normal_json_records_list_list.append(flatten_json_records_list)
    #break

In [199]:
len(normal_json_records_list_list)

1279

# Output data table to CSV file 

In [200]:
col_name_on_offset = get_df_header_of_each_col(len(wanted_packets), json_records_list_list, []) 

In [201]:
# cast the IceId record into one record 
Hancitor_df = make_list_json_record_into_one_record(col_name_on_offset, json_records_list_list, len(wanted_packets))
Hancitor_df.to_csv('Hancitor_only_data_set.csv', index=False)
Hancitor_df

,tcp_tcp.checksum.status_0,tcp_tcp.urgent_pointer_0,eth_eth.dst_tree_eth.lg_0,frame_frame.marked_0,ip_ip.frag_offset_0,frame_frame.time_epoch_0,http_http.host_0,ip_ip.flags_tree_ip.flags.mf_0,ip_ip.flags_0,ip_ip.flags_tree_ip.flags.df_0,...,http_GET /15s.bin HTTP/1.1\r\n_http.request.method_4,http_http.next_request_in_4,http_GET /0102s.bin HTTP/1.1\r\n_http.request.version_4,ip_ip.dst_4,tcp_tcp.analysis_tcp.analysis.bytes_in_flight_4,http_GET /24s.bin HTTP/1.1\r\n_http.request.uri_4,http_GET /107.bin HTTP/1.1\r\n__ws.expert__ws.expert.severity_4,frame_frame.encap_type_4,http_GET /1305s.bin HTTP/1.1\r\n__ws.expert__ws.expert.group_4,eth_eth.type_4
0,2.0,0.0,0.0,0.0,0.0,1.612283e+09,16.0,0.0,2.0,1.0,...,0.0,65.0,13.0,1.479348e+08,357.0,0.0,0.0,1.0,0.0,2048.0
1,2.0,0.0,0.0,0.0,0.0,1.612891e+09,16.0,0.0,2.0,1.0,...,0.0,0.0,0.0,1.478519e+08,353.0,0.0,0.0,1.0,0.0,2048.0
2,2.0,0.0,0.0,0.0,0.0,1.616079e+09,16.0,0.0,2.0,1.0,...,0.0,105.0,0.0,1.478736e+08,343.0,0.0,0.0,1.0,0.0,2048.0
3,2.0,0.0,0.0,0.0,0.0,1.617722e+09,16.0,0.0,2.0,1.0,...,0.0,65.0,0.0,1.478736e+08,335.0,0.0,0.0,1.0,0.0,2048.0
4,2.0,0.0,0.0,0.0,0.0,1.620943e+09,16.0,0.0,2.0,1.0,...,0.0,105.0,0.0,1.480473e+08,339.0,0.0,0.0,1.0,10.0,2048.0
5,2.0,0.0,0.0,0.0,0.0,1.621530e+09,16.0,0.0,2.0,1.0,...,0.0,105.0,0.0,1.480473e+08,337.0,0.0,0.0,1.0,0.0,2048.0
6,2.0,0.0,0.0,0.0,0.0,1.621869e+09,16.0,0.0,2.0,1.0,...,0.0,105.0,0.0,1.480473e+08,337.0,179.0,0.0,1.0,0.0,2048.0
7,2.0,0.0,0.0,0.0,0.0,1.622560e+09,16.0,0.0,2.0,1.0,...,0.0,105.0,0.0,1.480473e+08,341.0,0.0,0.0,1.0,0.0,2048.0
8,2.0,0.0,0.0,0.0,0.0,1.623770e+09,16.0,0.0,2.0,1.0,...,11.0,105.0,0.0,1.479454e+08,339.0,0.0,0.0,1.0,0.0,2048.0
9,2.0,0.0,0.0,0.0,0.0,1.623948e+09,16.0,0.0,2.0,1.0,...,0.0,0.0,0.0,1.479454e+08,341.0,0.0,0.0,1.0,0.0,2048.0


In [202]:
# cast the normal record into one record 
normal_df = make_list_json_record_into_one_record(col_name_on_offset, normal_json_records_list_list, len(wanted_packets))
normal_df.to_csv('normal_Hancitor_only_data_set.csv', index=False)
normal_df

,tcp_tcp.checksum.status_0,tcp_tcp.urgent_pointer_0,eth_eth.dst_tree_eth.lg_0,frame_frame.marked_0,ip_ip.frag_offset_0,frame_frame.time_epoch_0,http_http.host_0,ip_ip.flags_tree_ip.flags.mf_0,ip_ip.flags_0,ip_ip.flags_tree_ip.flags.df_0,...,http_GET /15s.bin HTTP/1.1\r\n_http.request.method_4,http_http.next_request_in_4,http_GET /0102s.bin HTTP/1.1\r\n_http.request.version_4,ip_ip.dst_4,tcp_tcp.analysis_tcp.analysis.bytes_in_flight_4,http_GET /24s.bin HTTP/1.1\r\n_http.request.uri_4,http_GET /107.bin HTTP/1.1\r\n__ws.expert__ws.expert.severity_4,frame_frame.encap_type_4,http_GET /1305s.bin HTTP/1.1\r\n__ws.expert__ws.expert.group_4,eth_eth.type_4
0,2.0,0.0,0.0,0.0,0.0,1.668627e+09,333.0,0.0,2.0,1.0,...,0.0,0.0,0.0,1.732703e+09,473.0,0.0,0.0,1.0,0.0,2048.0
1,2.0,0.0,0.0,0.0,0.0,1.668627e+09,367.0,0.0,2.0,1.0,...,0.0,0.0,0.0,3.770000e+02,371.0,0.0,0.0,369.0,0.0,375.0
2,376.0,375.0,372.0,372.0,375.0,3.700000e+02,367.0,372.0,376.0,372.0,...,0.0,0.0,0.0,3.770000e+02,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,0.0,0.0,0.0,0.0,1.668627e+09,382.0,0.0,2.0,1.0,...,0.0,0.0,0.0,3.770000e+02,371.0,0.0,0.0,369.0,0.0,375.0
4,376.0,375.0,372.0,372.0,375.0,3.700000e+02,382.0,372.0,376.0,372.0,...,0.0,0.0,0.0,3.770000e+02,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,2.0,0.0,0.0,0.0,0.0,1.668631e+09,2997.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
1275,376.0,375.0,372.0,372.0,0.0,3.700000e+02,2997.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,371.0,0.0,0.0,369.0,0.0,375.0
1276,2.0,0.0,0.0,0.0,0.0,1.668631e+09,2380.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
1277,376.0,375.0,372.0,372.0,375.0,3.700000e+02,2380.0,372.0,376.0,372.0,...,0.0,0.0,0.0,0.000000e+00,371.0,0.0,0.0,369.0,0.0,375.0


In [203]:
# output flated data and str map to files
with open('Hancitor_record_str_to_int_map.json', 'w') as fp:
    json.dump(record_str_to_int_map, fp)

# Free and testing space, do not run the following 

In [ ]:
wire_shark_type = set()
for key,val in ProjectLookUpTable.wire_shark_type_lookup_map.items():
    wire_shark_type.add(val)
wire_shark_type

In [ ]:

wire_shark_type_lookup_map = {}
wire_shark_description_lookup_map = {}
for i in range(len(field_list)):
    wire_shark_type_lookup_map[field_list[i]] = type_list[i]
    wire_shark_description_lookup_map[field_list[i]] = description_list[i]
